# Exemplo 12: Dados Contínuos (Streaming)
## Contagem contínua de palavras

Spark Streaming is an extension of the Spark API that enables stream processing of live data streams. A Stream is a continuous source of data generated in real time. Data can be ingested from many sources like filesystem, Kafka, Flume, Kinesis, or TCP sockets, and can be processed using complex algorithms like map, reduce, join and window, and apply Spark’s machine learning or graph processing algorithms on continuous data streams. Finally, processed data can be pushed out to filesystems, databases, and dashboards.

This example counts words in text encoded with UTF8 received from the network every second.

The script *streaming_server.py* reads a text file and send one line per second and create a TCP server that Spark Streaming would connect to receive data.

Usage: **./streaming_server.py \<filename> \<port>**, where *filename* ia a UTF-8 text file and *port* is a port number between 9700 and 9800 that should be configured in the configuration parameters section.

The notebook process the word count in each reading or during a slide windows.

In [1]:
# Start Spark environment
import findspark
findspark.init()

In [2]:
# Load Spark Library
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext

import time, os, re

### Configuration parameters

In [3]:
#Streaming server port

port=9763

## Create Spark Context and Streming Context

In [4]:
# Create Spark Context
sc = SparkContext("local[*]","Stream_WordCount") \
     .getOrCreate()

# Create Streming Context reading server each 1 sec
ssc = StreamingContext(sc, 1)

#  Set checkpoint directory
ssc.checkpoint("/tmp/spark-checkpoint")

## Read text received by Socket

In [5]:
# Create a DStream that will connect to hostname:port
lines = ssc.socketTextStream("localhost", port)

# Print line read
lines.pprint()

# Split each line into words
words = lines.flatMap(lambda line: line.split())

## Count words in each read

In [6]:
# Count each word in each line
pairs = words.map(lambda word: (word, 1))

# Count the number of each word in lines
wordCounts = pairs.reduceByKey(lambda x, y: x + y)

# Print the first ten elements of each line
#wordCounts.pprint()

## Count words in each window

In [7]:
# Count the total words last 30 seconds of data, every 10 seconds
# reduceByKeyAndWindow(oper1, oper2,windows_time,result_time)
# oper1: operation in sliding window
# oper2: operation to remove last window
# windows_time: window length (total time of window)
#result_time: time when the result is evalutated

# Count the number of each word in windows of 30 sec showing at 10 sec
windowedWordCounts = wordCounts.reduceByKeyAndWindow(lambda x, y: x + y, lambda x, y: x - y, 30, 10)

# Print the first ten elements of each window period
windowedWordCounts.pprint()

## Start Sreaming processing

In [ ]:
ssc.start()             # Start the computation
ssc.awaitTermination()  # Wait for the computation to terminate

-------------------------------------------
Time: 2020-04-12 16:54:36
-------------------------------------------

-------------------------------------------
Time: 2020-04-12 16:54:37
-------------------------------------------
                                 apache license

-------------------------------------------
Time: 2020-04-12 16:54:38
-------------------------------------------
                           version 20 january 2004

-------------------------------------------
Time: 2020-04-12 16:54:39
-------------------------------------------
                        httpwwwapacheorglicenses

-------------------------------------------
Time: 2020-04-12 16:54:40
-------------------------------------------


-------------------------------------------
Time: 2020-04-12 16:54:41
-------------------------------------------
   terms and conditions for use reproduction and distribution

-------------------------------------------
Time: 2020-04-12 16:54:42
-----------------------------